<a href="https://colab.research.google.com/github/walkerjian/dev/blob/main/Speculative_RAG_Enhancing_retrieval_augmented_generation_through_drafting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To implement the **Speculative Retrieval-Augmented Generation (Speculative RAG)** in a platform like Google Colab, follow these steps:

### 1. **Set Up the Environment**
   - Start with setting up your environment, ensuring all necessary libraries and frameworks are installed.

   ```python
   !pip install torch transformers sentence-transformers scikit-learn
   ```

### 2. **Load Pre-trained Models**
   - You need two models: a smaller specialist LM (`MDrafter`) and a larger generalist LM (`MVerifier`). You can use models from Hugging Face.

   ```python
   from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

   # Load the drafter model (small specialist model)
   drafter_model_name = "facebook/bart-large"
   drafter_model = AutoModelForSeq2SeqLM.from_pretrained(drafter_model_name)
   drafter_tokenizer = AutoTokenizer.from_pretrained(drafter_model_name)

   # Load the verifier model (larger generalist model)
   verifier_model_name = "google/t5-large-lm-adapt"
   verifier_model = AutoModelForSeq2SeqLM.from_pretrained(verifier_model_name)
   verifier_tokenizer = AutoTokenizer.from_pretrained(verifier_model_name)
   ```

### 3. **Document Retrieval and Clustering**
   - Retrieve relevant documents and cluster them using an embedding model like `sentence-transformers`.

   ```python
   from sentence_transformers import SentenceTransformer
   from sklearn.cluster import KMeans

   embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

   # Assume `retrieved_docs` is a list of retrieved documents
   doc_embeddings = embedding_model.encode(retrieved_docs)

   # Cluster the documents
   num_clusters = 5  # This can be tuned
   kmeans = KMeans(n_clusters=num_clusters, random_state=42)
   kmeans.fit(doc_embeddings)
   clusters = kmeans.labels_

   # Create subsets from each cluster
   doc_subsets = []
   for i in range(num_clusters):
       cluster_docs = [retrieved_docs[j] for j in range(len(retrieved_docs)) if clusters[j] == i]
       doc_subsets.append(cluster_docs)
   ```

### 4. **Draft Generation with MDrafter**
   - For each subset of documents, generate drafts using the drafter model.

   ```python
   drafts = []
   for subset in doc_subsets:
       input_text = " ".join(subset)
       inputs = drafter_tokenizer(input_text, return_tensors="pt", truncation=True, padding=True)
       outputs = drafter_model.generate(**inputs)
       draft = drafter_tokenizer.decode(outputs[0], skip_special_tokens=True)
       drafts.append(draft)
   ```

### 5. **Verification with MVerifier**
   - Use the verifier model to evaluate each draft and select the most reliable one.

   ```python
   best_draft = None
   best_score = float('-inf')

   for draft in drafts:
       verifier_input = verifier_tokenizer(draft, return_tensors="pt")
       verifier_output = verifier_model.generate(**verifier_input)
       score = verifier_model(**verifier_input, labels=verifier_input["input_ids"]).loss.item()
       
       if score > best_score:
           best_draft = draft
           best_score = score
   ```

### 6. **Final Answer Selection**
   - The best draft with the highest score is selected as the final answer.

   ```python
   print("Best Draft:", best_draft)
   ```

### 7. **Optimizations and Scaling**
   - Implement parallel processing or multi-threading to handle larger datasets and speed up the process, particularly during the drafting stage.
   - Consider using Tensor Parallelism if working with extremely large models to distribute the computational load across multiple GPUs.

### 8. **Further Customization**
   - You may modify the scoring mechanism to include more sophisticated metrics or incorporate additional model layers for enhanced reasoning capabilities.
   - Fine-tune the `MDrafter` model for specific types of queries or integrate specialized token-level adjustments for better rationale generation.

### 9. **Testing and Validation**
   - Test the entire pipeline on various datasets like TriviaQA, PubHealth, etc., to validate performance improvements in both accuracy and latency.

This setup in Google Colab should allow you to effectively implement the Speculative RAG framework. Adjustments might be necessary based on the specific dataset or the computational resources available.

##i like this, and will implement it, but i also want a real world example to sink its' teeth into, such as how to generate essays that are undetectable by any ai detector, have a think abut that



To create essays that are difficult for AI detectors to flag as AI-generated, you can leverage the Speculative RAG approach with some real-world customization. Here's a step-by-step guide on how to apply this in a practical context:

### 1. **Problem Formulation**
   - **Goal**: Generate essays that blend high-quality, human-like content with factual accuracy while avoiding detection by AI detectors.
   - **Approach**: Utilize Speculative RAG to draft multiple versions of essay paragraphs using different sources and perspectives, then refine and combine the best segments into a final essay.

### 2. **Setting Up the Environment**
   - Use the same setup as described earlier with the `drafter_model` and `verifier_model`. Additionally, include a style-checking model or heuristic to assess and modify the tone, structure, and language complexity.

### 3. **Document Retrieval and Clustering**
   - Retrieve diverse sets of documents related to the essay topic from reliable sources (e.g., academic papers, books, articles). Cluster these documents into different themes or perspectives.

   ```python
   essay_topic = "The impact of climate change on global agriculture"
   retrieved_docs = retrieve_documents(essay_topic)  # Custom function to fetch relevant docs
   doc_embeddings = embedding_model.encode(retrieved_docs)

   # Cluster the documents into themes like "economic impact", "environmental consequences", "policy changes", etc.
   num_clusters = 4
   kmeans = KMeans(n_clusters=num_clusters, random_state=42)
   kmeans.fit(doc_embeddings)
   clusters = kmeans.labels_

   doc_subsets = []
   for i in range(num_clusters):
       cluster_docs = [retrieved_docs[j] for j in range(len(retrieved_docs)) if clusters[j] == i]
       doc_subsets.append(cluster_docs)
   ```

### 4. **Draft Generation with a Human-like Style**
   - Focus on generating content that mimics human writing styles, including varied sentence structures, nuanced language, and a mix of formal and informal tones.

   ```python
   drafts = []
   for subset in doc_subsets:
       input_text = " ".join(subset)
       inputs = drafter_tokenizer(input_text, return_tensors="pt", truncation=True, padding=True)
       outputs = drafter_model.generate(**inputs)
       draft = drafter_tokenizer.decode(outputs[0], skip_special_tokens=True)
       
       # Apply style modifications to make it more human-like
       humanized_draft = enhance_human_style(draft)  # Custom function to tweak style
       drafts.append(humanized_draft)
   ```

### 5. **Verification with Human-Like Consistency**
   - Use the verifier model to evaluate each draft, focusing on human-like coherence, consistency, and originality. You might also integrate a paraphrasing step to further differentiate drafts.

   ```python
   best_draft = None
   best_score = float('-inf')

   for draft in drafts:
       verifier_input = verifier_tokenizer(draft, return_tensors="pt")
       verifier_output = verifier_model.generate(**verifier_input)
       score = verifier_model(**verifier_input, labels=verifier_input["input_ids"]).loss.item()
       
       # Incorporate additional checks for human-like qualities
       score += evaluate_human_likeness(draft)  # Custom function for evaluating style and originality
       
       if score > best_score:
           best_draft = draft
           best_score = score
   ```

### 6. **Final Assembly**
   - Combine the best parts of each draft into a cohesive essay, ensuring variation in style and content to evade detection. Make manual tweaks if necessary to add unique insights or subtle errors that AI might miss.

   ```python
   final_essay = combine_drafts(drafts)  # Custom function to merge the best parts of drafts
   print("Final Essay:", final_essay)
   ```

### 7. **Testing Against AI Detectors**
   - Before finalizing the essay, run it through various AI detection tools to identify and adjust any parts that might trigger flags. Use this feedback loop to refine your approach further.

   ```python
   detection_score = check_ai_detection(final_essay)  # Custom function to simulate AI detection checks
   while detection_score > threshold:
       final_essay = further_humanize(final_essay)  # Custom function for additional tweaks
       detection_score = check_ai_detection(final_essay)
   ```

### 8. **Deploy and Iterate**
   - Use this method to generate essays on various topics, iterating on the process and refining your models to consistently produce high-quality, undetectable content.

### **Real-World Considerations**
   - **Ethics**: Consider the ethical implications of generating undetectable AI-generated content, especially in academic and professional contexts.
   - **Legal**: Be aware of potential legal ramifications of using AI to produce content that is intended to evade detection, particularly in situations where originality is paramount.

This approach not only applies the Speculative RAG framework to a practical, real-world problem but also opens up avenues for more sophisticated content generation strategies.

In [13]:
!pip install torch transformers sentence-transformers scikit-learn openai

In [1]:
!pip install openai==0.28


In [5]:
from google.colab import userdata
import openai

# Retrieve the OpenAI API key stored as a secret
openai.api_key = userdata.get('OPENAI_API_KEY')

# Verify the key is loaded
if openai.api_key is None:
    raise ValueError("API key not found. Ensure OPENAI_API_KEY is set as a secret in Colab.")

# Function to fetch documents using the OpenAI API using chat completions
def fetch_documents_from_openai(query):
    response = openai.ChatCompletion.create(
        model="gpt-4",  # Specify the chat model, like gpt-4
        messages=[
            {"role": "system", "content": "You are a research assistant."},
            {"role": "user", "content": query}
        ],
        max_tokens=500
    )
    return response.choices[0].message['content'].strip()

# Define research queries for testing
queries = [
    "Provide an overview of recent research on the impact of climate change on global agriculture.",
    "Summarize papers discussing the economic consequences of climate change.",
    "Explain policies that address climate change's effect on agriculture."
]

# Fetch documents for each query
retrieved_docs = [fetch_documents_from_openai(query) for query in queries]

# Display the documents
for idx, doc in enumerate(retrieved_docs):
    print(f"Document {idx+1}:\n{doc}\n")


Document 1:
Various studies conducted recently highlight the significant impact of climate change on global agriculture and food security. 

1. Yield Fluctuation: A study by Ray et al. (2019) indicates that climate change is responsible for an average yield reduction of 1%-2% per decade over the past century for the world's top crops such as maize, wheat, and barley. 

2. Impact on Nutrition: Climate change affects not only the quantity but also the quality of the crops. A research paper by Myers et al. (2014) found that rising CO2 levels are associated with reduced protein and micronutrient contents in key crops like rice and wheat, worsening nutritional security.

3. Shift in Crop Patterns: Studies, such as those by Zhao et al. (2017) and Zabel et al. (2019), indicate that climate change is leading to a shift in agricultural zones, with warmer temperatures allowing farming in previously cold regions, while adversely affecting crop productivity in hotter climates. 

4. Increased Pest 

In [6]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Load the drafter model (small specialist model)
drafter_model_name = "facebook/bart-large"
drafter_model = AutoModelForSeq2SeqLM.from_pretrained(drafter_model_name)
drafter_tokenizer = AutoTokenizer.from_pretrained(drafter_model_name)

# Load the verifier model (larger generalist model)
verifier_model_name = "google/t5-large-lm-adapt"
verifier_model = AutoModelForSeq2SeqLM.from_pretrained(verifier_model_name)
verifier_tokenizer = AutoTokenizer.from_pretrained(verifier_model_name)


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [8]:
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans

embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

# Assume `retrieved_docs` is a list of retrieved documents
doc_embeddings = embedding_model.encode(retrieved_docs)

# Cluster the documents
# Change the number of clusters to be less than or equal to the number of documents
num_clusters = min(3, len(retrieved_docs))  # This ensures num_clusters <= n_samples
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
kmeans.fit(doc_embeddings)
clusters = kmeans.labels_

# Create subsets from each cluster
doc_subsets = []
for i in range(num_clusters):
    cluster_docs = [retrieved_docs[j] for j in range(len(retrieved_docs)) if clusters[j] == i]
    doc_subsets.append(cluster_docs)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [9]:
drafts = []
for subset in doc_subsets:
    input_text = " ".join(subset)
    inputs = drafter_tokenizer(input_text, return_tensors="pt", truncation=True, padding=True)
    outputs = drafter_model.generate(**inputs)
    draft = drafter_tokenizer.decode(outputs[0], skip_special_tokens=True)
    drafts.append(draft)


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [10]:
best_draft = None
best_score = float('-inf')

for draft in drafts:
    verifier_input = verifier_tokenizer(draft, return_tensors="pt")
    verifier_output = verifier_model.generate(**verifier_input)
    score = verifier_model(**verifier_input, labels=verifier_input["input_ids"]).loss.item()

    if score > best_score:
        best_draft = draft
        best_score = score


In [11]:
print("Best Draft:", best_draft)


Best Draft: Climate change significantly impacts agriculture through changes in temperature, precipitation patterns, and extreme weather events


In [14]:
essay_topic = "The impact of climate change on global agriculture"
retrieved_docs = fetch_documents_from_openai(essay_topic)  # Custom function to fetch relevant docs

# Ensure retrieved_docs is a list of strings (documents)
# If it's a single string, split it into sentences or paragraphs
if isinstance(retrieved_docs, str):
    # Example: Splitting into sentences using nltk
    import nltk
    nltk.download('punkt')
    retrieved_docs = nltk.sent_tokenize(retrieved_docs)

doc_embeddings = embedding_model.encode(retrieved_docs)

# Cluster the documents into themes like "economic impact", "environmental consequences", "policy changes", etc.
num_clusters = 4
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
kmeans.fit(doc_embeddings)
clusters = kmeans.labels_

doc_subsets = []
for i in range(num_clusters):
    cluster_docs = [retrieved_docs[j] for j in range(len(retrieved_docs)) if clusters[j] == i]
    doc_subsets.append(cluster_docs)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [18]:
# Function to fetch synonyms using OpenAI's Chat API
def get_synonyms(word):
    response = openai.ChatCompletion.create(
        model="gpt-4",  # Adjust depending on your access
        messages=[
            {"role": "system", "content": "You are a helpful assistant. Provide synonyms for words."},
            {"role": "user", "content": f"What are some synonyms for the word '{word}' that sound natural and human-like?"}
        ],
        max_tokens=50,
        temperature=0.5
    )
    # Extracting text from the response
    text = response.choices[0].message['content'].strip()
    # Assume the response lists synonyms in a comma-separated format
    synonyms = text.split(', ')
    return synonyms


In [19]:
import random
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
nltk.download('punkt')

def enhance_human_style(text):
    sentences = sent_tokenize(text)
    enhanced_sentences = []

    for sentence in sentences:
        words = word_tokenize(sentence)
        enhanced_words = []

        for word in words:
            # Replace words with synonyms at random with a controlled probability
            if random.random() < 0.3:  # 30% chance to replace a word with its synonym
                synonyms = get_synonyms(word)
                if synonyms:
                    # Select a random synonym to replace the word
                    enhanced_words.append(random.choice(synonyms))
                else:
                    enhanced_words.append(word)
            else:
                enhanced_words.append(word)

        # Reconstruct the sentence
        enhanced_sentence = ' '.join(enhanced_words)
        enhanced_sentences.append(enhanced_sentence)

    # Join all enhanced sentences into the final text
    enhanced_text = ' '.join(enhanced_sentences)
    return enhanced_text


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [20]:
drafts = []
for subset in doc_subsets:
    input_text = " ".join(subset)
    inputs = drafter_tokenizer(input_text, return_tensors="pt", truncation=True, padding=True)
    outputs = drafter_model.generate(**inputs)
    draft = drafter_tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Apply style modifications to make it more human-like
    humanized_draft = enhance_human_style(draft)  # Custom function to tweak style
    drafts.append(humanized_draft)

In [30]:
def evaluate_human_likeness(text):
    # This function will use OpenAI's API to evaluate text
    # Use openai.ChatCompletion.create for chat models like gpt-4
    response = openai.ChatCompletion.create(
        model="gpt-4",  # or "gpt-4" if you have access
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},  # system message
            {"role": "user", "content": f"Rate how natural and human-like the following text sounds: \"{text}\". Give a score from 0 to 10."}  # user message
        ],
        max_tokens=10,
        temperature=0
    )
    # Extract the content from the message
    response_text = response.choices[0].message['content'].strip()
    # Find numeric values in the response
    import re
    match = re.search(r'\d+', response_text)
    if match:
        score = float(match.group(0))
    else:
        score = 0  # Default score if no numbers are found
    return score


In [31]:
best_draft = None
best_score = float('-inf')

for draft in drafts:
    verifier_input = verifier_tokenizer(draft, return_tensors="pt")
    verifier_output = verifier_model.generate(**verifier_input)
    score = verifier_model(**verifier_input, labels=verifier_input["input_ids"]).loss.item()

    # Incorporate additional checks for human-like qualities
    score += evaluate_human_likeness(draft)  # Custom function for evaluating style and originality

    if score > best_score:
        best_draft = draft
        best_score = score


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [37]:
def combine_drafts1(drafts):
    """
    This function takes a list of drafts and combines them into a single final essay.
    You can implement your logic here to select the best parts of each draft
    or simply concatenate them together.
    """
    # This is a simple example that just concatenates the drafts:
    final_essay = " ".join(drafts)

    # More sophisticated logic can be added here to select and combine the best parts of each draft
    # For instance, you could use sentence similarity, keyword extraction, etc.
    # to identify and merge the most relevant content from each draft.

    return final_essay

In [36]:
def combine_drafts(drafts):
    """
    Use GPT-4 to combine multiple drafts into a coherent final essay.
    """
    prompt = "Combine the following drafts into a coherent essay: " + " BREAK ".join(drafts)
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are a skilled writer and editor."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=1024,  # Adjust based on the length needed
        temperature=0.5
    )
    final_essay = response.choices[0].message['content'].strip()
    return final_essay


In [38]:
final_essay = combine_drafts(drafts)  # Custom function to merge the best parts of drafts
print("Final Essay:", final_essay)


Final Essay: Climate change has a profound influence on our world, first and foremost through alterations in temperature, changes in precipitation, and the absorption of carbon dioxide. This influence is not only significant but also multifaceted. One of the most prominent areas affected is global agriculture and food production, a sector that is highly sensitive to the environment.

An increase in global temperature directly affects the science of plant biology, impacting the greenery on our planet. This rise in heat can result in a decrease in crop yields, posing a serious threat to food security worldwide. Additionally, alterations in precipitation patterns can cause droughts or floods, both of which are detrimental to agriculture.

Moreover, the absorption of carbon dioxide plays a crucial role in this context. Plants absorb carbon dioxide for photosynthesis, a process fundamental to their growth. However, an excess of this gas due to human activities can lead to global warming, fu

In [44]:
import re

def check_ai_detection(text):
    """
    Use GPT-4 to evaluate if the text sounds like it was written by a human.
    """
    prompt = f"Does the following text sound like it was written by a human or generated by AI? Provide a likelihood score from 0 to 10: \"{text}\""
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are an AI detection expert."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=50,
        temperature=0
    )
    score_text = response.choices[0].message['content'].strip()
    # Extract numerical score
    match = re.search(r'\d+', score_text)
    score = float(match.group(0)) if match else 10  # Default to a high detection score if unclear
    return score


In [45]:
def further_humanize(text):
    # Example adjustments to humanize text
    adjustments = [
        "Let's consider", "Interestingly enough,", "On the other hand,"
    ]
    sentences = text.split(". ")
    enhanced_sentences = [f"{random.choice(adjustments)} {sentence}" if random.random() < 0.2 else sentence for sentence in sentences]
    return ". ".join(enhanced_sentences)


In [46]:
threshold = 5  # Set a threshold for the detection score
final_essay = combine_drafts(drafts)

detection_score = check_ai_detection(final_essay)
while detection_score > threshold:
    final_essay = " ".join(further_humanize(final_essay))  # Implement further_humanize to adjust text style
    detection_score = check_ai_detection(final_essay)


RateLimitError: Request too large for gpt-4 in organization org-eneUUpqPCDVGqgpMR0BmDusj on tokens per min (TPM): Limit 80000, Requested 119574. The input or output tokens must be reduced in order to run successfully. Visit https://platform.openai.com/account/rate-limits to learn more.

In [40]:
detection_score = check_ai_detection(final_essay)  # Custom function to simulate AI detection checks
while detection_score > threshold:
    final_essay = further_humanize(final_essay)  # Custom function for additional tweaks
    detection_score = check_ai_detection(final_essay)


NameError: name 're' is not defined

In [47]:
import time
import random

def check_ai_detection(text):
    """
    Use GPT-4 to evaluate if the text sounds like it was written by a human.
    Split text into smaller parts if necessary to stay within token limits.
    """
    try:
        prompt = f"Does the following text sound like it was written by a human or generated by AI? \"{text}\""
        response = openai.ChatCompletion.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": "You are an AI detection expert."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=50,
            temperature=0
        )
        score_text = response.choices[0].message['content'].strip()
        match = re.search(r'\d+', score_text)
        score = float(match.group(0)) if match else 10  # Default to a high score if no numeric response
        return score
    except openai.RateLimitError:
        print("Rate limit exceeded, waiting to retry...")
        time.sleep(60)  # Wait 60 seconds before retrying
        return check_ai_detection(text)

def further_humanize(text):
    # Apply local transformations without needing additional API calls
    adjustments = ["Indeed,", "Furthermore,", "Moreover,", "However,"]
    sentences = text.split(". ")
    enhanced_sentences = [f"{random.choice(adjustments)} {sentence}" if random.random() < 0.2 else sentence for sentence in sentences]
    return ". ".join(enhanced_sentences).replace(" .", ".")

# Example usage
final_essay = combine_drafts(drafts)  # Assuming combine_drafts is optimized for token usage
detection_score = check_ai_detection(final_essay)
while detection_score > threshold:
    final_essay = further_humanize(final_essay)  # Use local changes to reduce API calls
    detection_score = check_ai_detection(final_essay)


AttributeError: module 'openai' has no attribute 'RateLimitError'